<h1 style="background:#6A9662; color:#FFF; padding:5px;">Marketing Analytics com Python</h1>

# <font color='#6A9662'>Definição de mix de produtos para otimizar lucro</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python usada neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.4


<h2 style="background:#6A9662; color:#FFF; padding:5px;">0. Definindo o problema de negócios</h2>

> A empresa *Lua Smart Techmonta* vende os modelos de smartphones, **Lua1**, **Lua2**, **Lua3**, **Lua4**, **Lua5**. A empresa está elaborando uma campanha *X* de marketing digital nas principais mídias sociais e precisa decidir quantas unidadesde de cada um dos modelos vai promover. Considerando que não há nenhum smartphone em estoque para esta campanha *X* e que após a camanha esses modelos serão atualizados e, por tanto, a empresa não quer manter nada em estoque.<br />
<mark> A *LuaSmart Tech* deseja saber quantas unidadesde de cada um de seus atuais modelos deve produzir (montar, testar e trabalhar na campanha de marketing *X*) para **maximizar seu lucro líquido**</mark> tendo como restrição não exceder mais horas de trabalho do que as disponíveis e também não deseja produzir mais do que pode vender pelos canais digitais; conforme seu plano de vendas elaborado pelo Departamento de Marketing:

|Modelo|Qtde máxima da<br> campanha ($N_i$)|Valor unitário <br> para venda ($V_i$)|
|:-----|--------------------------------:|-------------------------------------:|
|Lua1|500|640|
|Lua2|600|790|
|Lua3|750|880|
|Lua4|900|950|
|Lua5|1200|1100|

<u>Nota:</u> a quantidade mínima para cada um dos modelos deve ser de **200 unidades**

#### Composição dos custos

O **custo** dos componentes, bem como o tempo para montagem e testes são os seguintes:

|Modelo|Custo total dos <br> componentes ($C_i$)|Horas para <br> montagem ($Hm_i$)|Horas para <br> teste ($Ht_i$)|
|:-----|---------------------------------------:|--------------------------------:|-----------------------------:|
|Lua1|330|5|1|
|Lua2|400|6|1|
|Lua3|440|7|2|
|Lua4|500|8|2|
|Lua5|620|9|3|

#### Restrições quanto a mão de obra necessária para a montagem e testes:

- Existem no **máximo 20.000 horas de montagem** e **4.000 horas de teste** disponíveis**. 
- Cada hora de trabalho para <mark>montagem custa R\$11</mark> para os modelos Lua1, Lua2 e Lua3 e <mark>R\$17</mark> para os modelos Lua4 e Lua5
- Cada hora de trabalho para <mark>teste custa R\$15</mark> para todos os modelos. 

### Objetivo do projeto

Realizar a **otimização de preços** e compor um mix dos produtos da *LuaSmart Tech*, isto é, o objetivo é **maximizar o lucro**

<h2 style="background:#6A9662; color:#FFF; padding:5px;">1. Instalando e carregando o pacotes PuLP</h2>

Resolver um problema de otimização não é um processo linear, mas o processo pode ser dividido em cinco etapas gerais:

1. Obtendo a descrição do problema
2. Formulando o programa matemático
3. Resolvendo o programa matemático
4. Executar algumas análises pós-ideais
5. Apresentando a solução e análise

No entanto, muitas vezes existem “ciclos de feedback” neste processo. Por exemplo, depois de formular e resolver um problema de otimização, você frequentemente desejará considerar a validade de sua solução (geralmente consultando a pessoa que forneceu a descrição do problema). Se sua solução for inválida, você pode precisar alterar ou atualizar sua formulação para incorporar seu novo entendimento do problema real. Este processo é mostrado no Diagrama de Metodologia de Pesquisa Operacional.
![title](https://coin-or.github.io/pulp/_images/or_methodology.jpg)

<u>Links úteis:</u>
- [pypi](https://pypi.org/project/PuLP/)
- [documentation](https://coin-or.github.io/pulp/)
- [pulp: Pulp classes](https://www.coin-or.org/PuLP/pulp.html)

In [2]:
# Imports
from pulp import *

In [3]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Wecchi Data Science" --iversions

Author: Wecchi Data Science

sys    : 3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
re     : 2.2.1
ctypes : 1.1.0
json   : 2.0.9
logging: 0.5.1.2
pulp   : 2.4



<h2 style="background:#6A9662; color:#FFF; padding:5px;">2. Criando o modelo matemático para a otimização</h2>

### <u>Parâmetros</u>

- **$N_{i}$** = Número máximo de smartphones modelo tipo i para venda, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}
- **$V_{i}$** = Preço de venda de smartphones modelo tipo i, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}
- **$C_{i}$** = Custo das peças e componentes para montagem do smartphones modelo tipo i, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}
- **$MOm_{i}$** = Custo de mão de obra de montagem por hora de smartphones modelo tipo i, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}
- **$MOt_{i}$** = Custo de mão de obra de teste por hora de smartphones modelo tipo i, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}
- **$Hm_{i}$** = Horas de montagem necessárias para construir cada modelo de smartphone tipo i, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}
- **$Ht_{i}$** = Horas de teste necessárias para testar cada modelo de smartphone tipo i, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}
- **$Hm$** = Número **máximo** de horas de trabalho de montagem
- **$Ht$** = Número **máximo** de horas de trabalho de teste

### <u>Variável de decisão</u>

- **$X_{i}$** = Número de smartphones modelo tipo i a produzir este mês, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}

### <u>Função Objetivo</u>: $Venda - MO_{montagem} - MO_{testes} - Custo_{peças}$

### $$Lucro=\sum_{i=1}^{5}(X_i \times V_i) - \sum_{i=1}^{5}(X_i \times MOm_i \times Hm_i) - \sum_{i=1}^{5}(X_i \times MOt_i \times Ht_i) - \sum_{i=1}^{5}(X_i \times C_i)$$

### <u>Restrições impostas pela campanha de Marketing</u>

- O número de smartphones modelo tipo i a serem produzidos não pode ser inferior a 200, ou seja, $X_i >= 200$, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}.

- O número total de horas de montagem - $Hm$ - não pode ser maior que o número máximo de horas de mão de obra de montagem disponíveis (**limitados a 20.000**).

- O número total de horas de teste - $Ht$ - não pode ser maior do que o máximo de horas de mão de obra de teste disponíveis (**limitados a 4.000**).

- O número de smartphones modelo tipo i a serem produzidos - $X_i$ - não pode ser maior do que o número máximo de smartphones modelo tipo i a serem vendidos no mês - $N_i$ -, onde i pertence ao conjunto {Lua1, Lua2, Lua3, Lua4, Lua5}, pois isso poderia acarretar num estoque que deve ser evitado por questões de custos e de depreciação do produto que será descontinuado.

- Considerações conforme a definição do *público alvo* estabelecida pelo Departamento de Marketing na campanha vigente:
    - O número de smartphones modelo tipo **Lua5** não pode ser maior que o dobro da soma dos modelos tipo **Lua1** e **Lua3**: $X_5 \le 2\times(X_1 + X_3)$
    - O número de smartphones modelo tipo **Lua4** deve ser metade do valor da soma das quantidaes para os modelos tipo **Lua2** e **Lua3**: $X_4 = \frac{(X_2 + X_3)}{2}$ 

<h2 style="background:#6A9662; color:#FFF; padding:5px;">3. Implementando o modelo matemático</h2>

### <u>Organizando os parâmetros</u>

In [4]:
# Número máximo de smartphones para vender na campanha
N_1 = 500; N_2 = 600
N_3 = 750; N_4 = 900
N_5 = 1200

In [5]:
# Preço de venda de smartphones 
V_1 = 640; V_2 = 790 
V_3 = 880; V_4 = 950
V_5 = 1100

In [6]:
# Preço de custo das peças componentes para smartphones
C_1 = 330; C_2 = 400
C_3 = 440; C_4 = 500
C_5 = 620

In [7]:
# Custo de mão de obra de montagem por hora de smartphones
MO_m1 = 11; MO_m2 = 11
MO_m3 = 11; MO_m4 = 17
MO_m5 = 17

In [8]:
# Custo de mão de obra de teste por hora de smartphones 
MO_t1 = 15; MO_t2 = 15
MO_t3 = 15; MO_t4 = 15
MO_t5 = 15

In [9]:
# Número máximo de horas de trabalho de montagem
Hm = 20000 

In [10]:
# Número máximo de horas de trabalho de teste
Ht = 4000 

In [11]:
# Horas de montagem necessárias para construir cada modelo de smartphone
H_m1 = 5; H_m2 = 6
H_m3 = 7; H_m4 = 8
H_m5 = 9

In [12]:
# Horas de teste necessárias para testar cada modelo de smartphone
H_t1 = 1; H_t2 = 1
H_t3 = 2; H_t4 = 2
H_t5 = 3

### <u>Criando variável para o Problema de Otimização</u>

In [13]:
# Variável para o problema: otimizar o MixProdutos
problema = LpProblem("MixProdutos", LpMaximize)

### <u>Definindo a variável de decisão ($X_i$) para cada um dos modelos de smartphone</u>

In [14]:
# Define as variáveis
## :param lowBound mínimo
## :param upBound máximo (None significa sem limite máximo)
## :param cat tipo de dado da variável
X_1 = LpVariable("Unidades Lua1", 200, N_1, LpInteger)
X_2 = LpVariable("Unidades Lua2", 200, N_2, LpInteger)
X_3 = LpVariable("Unidades Lua3", 200, N_3, LpInteger)
X_4 = LpVariable("Unidades Lua4", 200, N_4, LpInteger)
X_5 = LpVariable("Unidades Lua5", 200, N_5, LpInteger)

### <u>Implementando a Função Objetivo</u>
### $$Lucro=\sum_{i=1}^{5}(X_i \times V_i) - \sum_{i=1}^{5}(X_i \times MOm_i \times Hm_i) - \sum_{i=1}^{5}(X_i \times MOt_i \times Ht_i) - \sum_{i=1}^{5}(X_i \times C_i)$$

In [15]:
# Venda: 1º componente da equação
venda = (X_1 * V_1) + (X_2 * V_2) + (X_3 * V_3) + (X_4 * V_4) + (X_5 * V_5)
venda

640*Unidades_Lua1 + 790*Unidades_Lua2 + 880*Unidades_Lua3 + 950*Unidades_Lua4 + 1100*Unidades_Lua5 + 0

In [16]:
# Custo de Montagem: 2º componente da equação
MO_montagem = (X_1 * MO_m1 * H_m1) + (X_2 * MO_m2 * H_m2) + (X_3 * MO_m3 * H_m3)\
+ (X_4 * MO_m4 * H_m4) + (X_5 * MO_m5 * H_m5)
MO_montagem

55*Unidades_Lua1 + 66*Unidades_Lua2 + 77*Unidades_Lua3 + 136*Unidades_Lua4 + 153*Unidades_Lua5 + 0

In [17]:
# Custo de Teste: 3º componente da equação
MO_teste = (X_1 * MO_t1 * H_t1) + (X_2 * MO_t2 * H_t2) + (X_3 * MO_t3 * H_t3)\
+ (X_4 * MO_t4 * H_t4) + (X_5 * MO_t5 * H_t5)
MO_teste

15*Unidades_Lua1 + 15*Unidades_Lua2 + 30*Unidades_Lua3 + 30*Unidades_Lua4 + 45*Unidades_Lua5 + 0

In [18]:
# Custo de Componentes: 4º componente da equação
Custo_pecas = (X_1 * C_1) + (X_2 * C_2)  + (X_3 * C_3)  + (X_4 * C_4)  + (X_5 * C_5) 
Custo_pecas

330*Unidades_Lua1 + 400*Unidades_Lua2 + 440*Unidades_Lua3 + 500*Unidades_Lua4 + 620*Unidades_Lua5 + 0

In [19]:
type(Custo_pecas)

pulp.pulp.LpAffineExpression

In [20]:
# Lucro = Venda - Custo de Montagem - Custo de Teste - Custo de Componentes
problema += venda - MO_montagem - MO_teste - Custo_pecas
problema

MixProdutos:
MAXIMIZE
240*Unidades_Lua1 + 309*Unidades_Lua2 + 333*Unidades_Lua3 + 284*Unidades_Lua4 + 282*Unidades_Lua5 + 0
VARIABLES
200 <= Unidades_Lua1 <= 500 Integer
200 <= Unidades_Lua2 <= 600 Integer
200 <= Unidades_Lua3 <= 750 Integer
200 <= Unidades_Lua4 <= 900 Integer
200 <= Unidades_Lua5 <= 1200 Integer

### <u> Implementando as restrições</u>

In [21]:
# Número máximo de horas de montagem
problema.addConstraint((X_1 * H_m1) + (X_2 * H_m2) + (X_3 * H_m3) + (X_4 * H_m4) + (X_5 * H_m5) <= Hm, 
                       name = "Número máximo de horas de montagem")

In [22]:
# Número máximo de horas de teste
problema.addConstraint((X_1 * H_t1) + (X_2 * H_t2) + (X_3 * H_t3) + (X_4 * H_t4) + (X_5 * H_t5) <= Ht,
                       name = "Número máximo de horas de teste")

In [23]:
# O número de smartphones modelo tipo Lua5 não pode ser maior que o dobro da soma dos modelos tipo Lua1 e Lua3 
problema.addConstraint(constraint = 2 * (X_1 + X_3) >= X_5,
                      name = "O número de Lua5 não pode ser maior que o dobro da soma dos tipo Lua1 e Lua3")

In [28]:
problema.addConstraint(constraint = (X_2 + X_3)/2 == X_4,
                      name = "O número de Lua4 deve ser metade do valor da soma das quantidaes para Lua2 e Lua3")

In [29]:
# Produção menor ou igual a demanda pelo modelo Lua1
problema += X_1 <= N_1,"Produção menor ou igual a demanda pelo modelo Lua1"

In [30]:
# Produção menor ou igual a demanda pelo modelo Lua2
problema += X_2 <= N_2,"Produção menor ou igual a demanda pelo modelo Lua2"

In [31]:
# Produção menor ou igual a demanda pelo modelo Lua3
problema += X_3 <= N_3,"Produção menor ou igual a demanda pelo modelo Lua3"

In [32]:
# Problema final
problema

MixProdutos:
MAXIMIZE
240*Unidades_Lua1 + 309*Unidades_Lua2 + 333*Unidades_Lua3 + 284*Unidades_Lua4 + 282*Unidades_Lua5 + 0
SUBJECT TO
Número_máximo_de_horas_de_montagem: 5 Unidades_Lua1 + 6 Unidades_Lua2
 + 7 Unidades_Lua3 + 8 Unidades_Lua4 + 9 Unidades_Lua5 <= 20000

Número_máximo_de_horas_de_teste: Unidades_Lua1 + Unidades_Lua2
 + 2 Unidades_Lua3 + 2 Unidades_Lua4 + 3 Unidades_Lua5 <= 4000

O_número_de_Lua5_não_pode_ser_maior_que_o_dobro_da_soma_dos_tipo_Lua1_e_Lua3:
 2 Unidades_Lua1 + 2 Unidades_Lua3 - Unidades_Lua5 >= 0

O_número_de_Lua4_deve_ser_metade_do_valor_da_soma_das_quantidaes_para_Lua2_e_Lua3:
 0.5 Unidades_Lua2 + 0.5 Unidades_Lua3 - Unidades_Lua4 = 0

Produção_menor_ou_igual_a_demanda_pelo_modelo_Lua1: Unidades_Lua1 <= 500

Produção_menor_ou_igual_a_demanda_pelo_modelo_Lua2: Unidades_Lua2 <= 600

Produção_menor_ou_igual_a_demanda_pelo_modelo_Lua3: Unidades_Lua3 <= 750

VARIABLES
200 <= Unidades_Lua1 <= 500 Integer
200 <= Unidades_Lua2 <= 600 Integer
200 <= Unidades_Lua3 

### <u>Resolvendo o problema de otimização do lucro com mix de produtos</u>

In [33]:
# Otimização - olhar o terminal para verificar se houve algum problema na execução
problema.solve()

1

Return status from solver:

|LpStatus key|string value|numerical value|
|:-----------|:----------:|--------------:|
|LpStatusOptimal|“Optimal”|1|
|LpStatusNotSolved|“Not Solved”|0|
|LpStatusInfeasible|“Infeasible”|-1|
|LpStatusUnbounded|“Unbounded”|-2|
|LpStatusUndefined|“Undefined”|-3|

In [34]:
# Lucro Maximizado
print("Lucro Maximizado:", value(problema.objective))

Lucro Maximizado: 715898.0


In [35]:
# Número de Unidades a serem produzidas de cada smartphone
for i in range(0,5):
    print("Modelo Lua{} :".format(i+1), problema.variables()[i].varValue)

Modelo Lua1 : 500.0
Modelo Lua2 : 598.0
Modelo Lua3 : 568.0
Modelo Lua4 : 583.0
Modelo Lua5 : 200.0


<h2 style="background:#6A9662; color:#FFF; padding:5px;">4. Conclusão</h2>

A empresa _Lua Smart Tech_ deve produzir as seguintes quantidades por tipo de smartphone:
- Modelo Lua1 : 500.0
- Modelo Lua2 : 598.0
- Modelo Lua3 : 568.0
- Modelo Lua4 : 583.0
- Modelo Lua5 : 200.0

Para atingir o **lucro máximo de: R\$ 715.898**